In [ ]:
!pip install simpy

# Priority resource

assembly workshop with one worker

2 types of jobs(priority[1]>priority[2])

In [ ]:
import simpy, random, numpy as np

jobs = {}
stats = {
    'cycle_times': {
        1: [],
        2: [],
    }
}

def job_arrival(env, workers):
  i = 0
  while True:
    i += 1
    yield env.timeout(random.expovariate(1/10)) #1/10mins
    job_prio = np.random.choice([1, 2], p=[.2, .8])
    name = i
    jobs[name] = env.now
    env.process(proc_job(env, name, job_prio, workers))

    if i == 1000:
      break

def proc_job(env, name, job_prio, workers):
  # print(len(workers.queue))
  with workers.request(priority=job_prio) as req:
    yield req
    yield env.timeout(20) #20mins

  # print(f'Job {name} with priority {job_prio} finished at {env.now}')
  jobs[name] = env.now - jobs[name]
  stats['cycle_times'][job_prio].append(jobs[name])


env = simpy.Environment()
workers = simpy.PriorityResource(env, capacity=1)
env.process(job_arrival(env, workers))
env.run()
print(f'Avg cycle time of job type 1: {np.mean(stats["cycle_times"][1]):.2f}')
print(f'Avg cycle time of job type 2: {np.mean(stats["cycle_times"][2]):.2f}')

Avg cycle time of job type 1: 36.69
Avg cycle time of job type 2: 6281.83
